In this notebook I intend to test a strategy where I invest an amount in NIFTY50 index and then rebalance the portfolio every week. The strategy is as follows:
- Invest INR 100,000- in NIFTY50 index
- If the return of the index is greater than X % in a week, sell the index to book the profit and take a long position again.
- Do this every Friday.

In [309]:
import yfinance as yf
import pandas as pd

In [310]:
SHORT_TERM_CAPITAL_GAINS_TAX_RATE = 0.20
BROKERAGE = 0
SECURITY_TRANSACTION_TAX_RATE = 0.001
TRANASCTION_CHARGES_RATE = 0.0000297
GST_RATE = 0.18
SEBI_CHARGES = 10
STAMP_DUTY_RATE = 0.00015 # Applicable only in case of buy transaction

def calculate_STCG(price, quantity, buy_price, sell_price):
    """
    Calculate the Short Term Capital Gains Tax
    """
    return max((sell_price - buy_price), 0) * quantity * SHORT_TERM_CAPITAL_GAINS_TAX_RATE

def calculate_charges(price, quantity, buy=True):
    """
    Calculate the charges for a transaction
    """
    if buy:
        stamp_duty = price * quantity * STAMP_DUTY_RATE
    else:
        stamp_duty = 0
    brokerage = BROKERAGE
    stt = price * quantity * SECURITY_TRANSACTION_TAX_RATE
    transaction_charges = price * quantity * TRANASCTION_CHARGES_RATE
    gst = (brokerage + SEBI_CHARGES + transaction_charges) * GST_RATE
    return brokerage + stt + transaction_charges + gst + SEBI_CHARGES + stamp_duty

In [ ]:
# Define the ticker symbol for NIFTY 50
ticker = "^NSEI"

# Set the time period
start_date = "2015-11-01"
end_date = "2025-01-02"

# Fetch the data
data = yf.download(ticker, start=start_date, end=end_date, interval='1d')
pd.DataFrame.drop(data, columns=['Close', 'Volume'], inplace=True)
data.rename(columns={'Adj Close': 'Close'}, inplace=True)
data = pd.concat([pd.DataFrame(data.iloc[0]).T, data.resample('W-FRI').last()])
# Add a column with default value set to 0
data['Returns'] = 0.0
data['SELL_Cost'] = 0.0
data['STCG'] = 0.0
data['BUY_Cost'] = 0.0
data['Profit'] = 0.0

In [312]:
data.loc[data.index[0], 'Close'] = data.loc[data.index[0], 'Open']
data['Returns'] = data['Close'].pct_change()

In [323]:
# Initial values
initial_amount = 1000000
amount = initial_amount

# Threshold percentage for selling (e.g., 2% return)
EXPECTED_RETURN = 0.15
THRESHOLD_PERCENTAGE = EXPECTED_RETURN / 100

In [324]:
buy_price = data.iloc[0]['Close']
quantity = initial_amount // buy_price

# Iterate over rows of a dataframe. If return is greater than threshold, sell the stock and buy again
for index, row in data[1:].iterrows():
    if row['Returns'] > THRESHOLD_PERCENTAGE:
        # Sell the stock
        sell_price = row['Close']
        stcg = calculate_STCG(row['Close'], quantity, buy_price, sell_price)
        charges = calculate_charges(row['Close'], quantity, buy=False)
        data.loc[index, 'SELL_Cost'] = charges
        data.loc[index, 'STCG'] = stcg
        
        # Buy again
        buy_price = row['Close']
        quantity = initial_amount // buy_price
        charges = calculate_charges(row['Close'], quantity, buy=True)
        data.loc[index, 'BUY_Cost'] = charges

        data.loc[index, 'Profit'] = initial_amount * row['Returns'] - row['SELL_Cost'] - row['STCG'] - row['BUY_Cost']

In [ ]:
profit = round(data['Profit'].sum(), 2)
print(f"Profit: INR {profit}/-")

In [ ]:
data